# Task 1: Binary Representations

In this section, It creates some basic bit manipulation functions.
Trying to rotate bits to the left

References from https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# rotl(x,n=1), rotating left that rotates the bits in a 32-bit unsigned integer to the left n places.



In [13]:
# First try at rotate left... but I missed something

def rotl(x, n=1):
    return (x << n) | (x >> n)

# Testing with a simple number that only has 1 bit on
print(bin(rotl(0b00000000000000000000000000000001, 1)))


0b10


does not rotate correctly.
shifted right by n, but to wrap the bits properly I should shift right by 32 - n. Also, forgot to make sure we stay within 32 bits .

In [ ]:
# Fixed it by wrapping the bits properly and keeping the result to 32 bits.
# ref on https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

def rotatel(x, n=1):

    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Testing again
print(bin(rotatel(0b00000000000000000000000000000001, 1)))  
print(bin(rotatel(0x80000000, 1)))  


0b10
0b1


# rotr (x,n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.





this is basically the opposite of the last one.
rotating bits to the right, and wrapping it back to the left

ref : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

In [ ]:
# First attempt at rotating right – but something feels off

def rotater(x, n=1):
    return (x >> n) | (x << n)

print(bin(rotater(0b00000000000000000000000000000010, 1)))  # expecting 0b1


0b101


In [22]:
# Fixed it 
def rotater(x, n=1):
   
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

print(bin(rotater(0b10, 1)))   
print(bin(rotater(0b1, 1)))    


0b1
0b10000000000000000000000000000000



the `return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF` function spins the bits to the right by n steps making sure everything stays within 32 bits  see https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb sections: bitwise shift , bitwise OR , bit masking , integer size

## ch ( x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

This function is like a little decision-maker.  
It looks at each bit of `x` — if it’s a 1, it picks the corresponding bit from `y`. If it’s a 0, it picks from `z`.

In [6]:
def ch(x, y, z):
    return (x & y) | (x & z)

print(bin(ch(0b1010, 0b1100, 0b0011))) # expecting 0b1010
print(bin(ch(0b1010, 0b1100, 0b0000))) # expecting 0b1000
print(bin(ch(0b1010, 0b0000, 0b0011))) # expecting 0b0000


0b1010
0b1000
0b10


In [7]:
# 
def ch(x, y, z):
   
    return (x & y) ^ (~x & z)

print(bin(ch(0b1010, 0b1100, 0b0011)))  # expecting 0b1010
print(bin(ch(0b1010, 0b1100, 0b0000)))  # expecting 0b1000
print(bin(ch(0b1010, 0b0000, 0b0011)))  # expecting 0b0000
print(bin(ch(0b1010, 0b1100, 0b1111)))  # expecting 0b1111

0b1001
0b1000
0b1
0b1101


The ch(x, y, z) function is based on bitwise operations covered in the Bitwise AND, OR, XOR, and NOT sections see ref: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb combining them to make a per-bit logic - "if x then y else z"

## maj(x, y, z)  which takes a majority vote of the bits in x, y, and z.
The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

This function is used in hashing algorithms to "vote" on each bit.
For each bit position `i`, it returns 1 if at least **two** of `x`, `y`, and `z` have a 1 in that position.

Example:  
- x = 1010  
- y = 1111  
- z = 0000  
→ maj = 1010 (only positions where at least two are 1)


In [ ]:
def maj(x, y, z): # majority function
    
    return x | y | z 

# Testing 
print(bin(maj(0b1010, 0b1100, 0b0011))) 


0b1111


In [ ]:
def maj(x, y, z):
    # Majority function: returns 1 if two or more of the bits are 1
    # and 0 otherwise.
    return (x & y) ^ (x & z) ^ (y & z) 

# Testing a few values:
x = 0b10101010 
y = 0b11110000
z = 0b00001111

print("x:", format(x, '08b')) 
print("y:", format(y, '08b'))
print("z:", format(z, '08b'))
print("maj:", format(maj(x, y, z), '08b'))


x: 10101010
y: 11110000
z: 00001111
maj: 10101010


### How the maj(x, y, z) logic works

The formula `(x & y) ^ (x & z) ^ (y & z)` works because it captures all the cases where **two or more** inputs have a 1.


- `x & y` → 1 only if both x and y have 1
- `x & z` → 1 if x and z do
- `y & z` → 1 if y and z do
Then XOR-ing all three gives us 1 in any position where **at least two** inputs have 1s.

results based on Bitwise (&) and XOR referenced in https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb 

- bitwise AND : talks how xx & y returns 1 only when both bits are 1.
- bitwise XOR : usefull when combining partial matches lin ( x & y ) ^ ( x & z) ^ (y & z).


# Task 2: Hash Functions

translation of a hash function from C into Python.



In [2]:
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


SyntaxError: invalid syntax (2215498579.py, line 1)

the goals is to 
- Convert the function to Python.

- Test it with a few strings.

- Explain why 31 and 101 are used.

- Include mistake version first, then correct version.

- Add natural markdown, comments, and commits.

In [5]:
def simple_hash(s):
    # Simple hash function that takes a string and returns a hash value.
    hashval = 0 # Initialize hash value to 0    
    for char in s: # Iterate over each character in the string
        hashval = hashval * 31 + ord(s) # Update hash value using the character's ASCII value
    return hashval % 101 # Return the hash value modulo 101 to keep it within a reasonable range
# Testing the simple_hash function
print(simple_hash("hello"))  


TypeError: ord() expected a character, but string of length 5 found